# Combining InSAR Data From Different Tracks and Orbits

## Setup

In [ ]:
%matplotlib inline
import glob
import numpy as np
import sys
import shutil
from itertools import product
from pathlib import Path
from skimage import morphology

import os
import matplotlib.pyplot as plt
from mintpy.utils import readfile, writefile, utils as ut
from mintpy import workflow, view, tsview, plot_network, plot_transection, plot_coherence_matrix

In [ ]:
cd ~/data/combined

In [ ]:
tracks = ['asc','desc']
years = [2018,2019,2020,2021]
# tracks = ['desc']
# years = [2020,2021]

## Set paths

In [ ]:
data_paths = {'asclower':Path('../asclower').resolve(),
             'ascupper':Path('../ascupper').resolve(),
             'desclower':Path('../desclower').resolve(),
             'descupper':Path('../descupper').resolve()}

# data_paths = {'desc_lower':Path('../desc_lower').resolve(),
#              'desc_upper':Path('../desc_upper').resolve()}

## Create eroded mask file and mask datasets

In [ ]:
for name in data_paths:
    print(name)
    p = data_paths[name]

    # mask_name = f'maskTempCoh_{name}_erd.h5'
    
    # Find files
    vels = list(p.glob('velocity*h5'))
    cohs = list(p.glob('temporalCoherence*h5'))
    tcoh_mask = p / f'maskTempCoh_{name}.h5'

    # Create new mask
    data, atr = readfile.read(tcoh_mask)
    erd = (morphology.binary_erosion(morphology.remove_small_holes(data,10), morphology.square(51))).astype(int)
    #     writefile.write({'mask':erd},mask_name,ref_file=tcoh_mask)
    
    for v in vels:
        name, ext = v.name.split('.')

        vel_name = f'{name}_mask.{ext}'
        vel, atr = readfile.read(v,datasetName='velocity')
        vel[erd == 0] = np.nan
        writefile.write({'velocity':vel},vel_name,ref_file=v)
        
        std_name = f'{name}_std_mask.{ext}'
        vel_std, atr = readfile.read(v,datasetName='velocityStd')
        vel_std[erd == 0] = np.nan
        writefile.write({'velocityStd':vel_std},std_name,ref_file=v)
        
    for c in cohs:
        name, ext = c.name.split('.')

        coh_name = f'{name}_mask.{ext}'
        coh, atr = readfile.read(c,datasetName='temporalCoherence')
        coh[erd == 0] = np.nan
        writefile.write({'temporalCoherence':coh},coh_name,ref_file=c)
        
    
#     # Mask velocity files
#     for v in vels:
#         name, ext = v.name.split('.')
#         vel_name = f'{name}_mask.{ext}'
#         if not Path(vel_name).exists():
#             !mask.py {str(v)} -m {mask_name} -o {vel_name}
    
#     # Mask coherence files
#     for c in cohs:
#         name, ext = c.name.split('.')
#         coh_name = f'{name}_mask.{ext}'
#         if not Path(coh_name).exists():
#             !mask.py {str(c)} -m {mask_name} -o {coh_name}

## Merge ascending and descending velocity by year

In [ ]:
# plotting results may need to be disabled at line 279 of image_stitch.py
for track, year in product(tracks,years):
    vel_pattern = f'velocity_{track}*{year}_mask.h5'
    vel_name = f'velocity_{track}{year}.h5'
    !image_stitch.py {vel_pattern} -o {vel_name} --no-offset
    
    std_pattern = f'velocity_{track}*{year}_std_mask.h5'
    std_name = f'std_{track}{year}.h5'
    !image_stitch.py {std_pattern} -o {std_name} --no-offset
    
    coh_pattern = f'temporalCoherence_{track}*{year}_mask.h5'
    coh_name = f'temporalCoherence_{track}{year}.h5'
    !image_stitch.py {coh_pattern} -o {coh_name} --no-offset

    # std_name = f'std_{track}{year}.h5'
    # !image_stitch.py {vel_pattern} -o {std_name} -d velocityStd --no-offset

    # temp_cohs = Path('..').glob(f'*/temporalCoherence_{track}*{year}.h5')
    # temp_cohs = ' '.join([str(x.resolve()) for x in temp_cohs])
    # temp_coh_name = f'temporalCoherence_{track}{year}.h5'
    # !image_stitch.py {temp_cohs} -o {temp_coh_name} --no-offset

    # !image_stitch.py {vel_pattern} -o {std_name} -d velocityStd --no-offset
    # !image_stitch.py {temp_cohs} -o {temp_coh_name} --no-offset

## Project from ascending/descending orbit to E-W/Up-Down

In [ ]:
for year in years:
    asc_vel = f'velocity_asc{year}.h5'
    desc_vel = f'velocity_desc{year}.h5'
    
    asc_std = f'std_asc{year}.h5'
    desc_std = f'std_desc{year}.h5'
    
    up_vel = f'velocity_up{year}.h5'
    ew_vel = f'velocity_ew{year}.h5'

    up_std = f'std_up{year}.h5'    
    ew_std = f'std_ew{year}.h5'

    !asc_desc2horz_vert.py {asc_vel} {desc_vel} -o {ew_vel} {up_vel} --max-ref-yx-diff 10000
    !asc_desc2horz_vert.py {asc_std} {desc_std} -o {ew_std} {up_std} --max-ref-yx-diff 10000

In [ ]:
for year in years:
    image1, image2 = f'temporalCoherence_asc{year}.h5 temporalCoherence_desc{year}.h5'.split(' ')
    out_name = f'temporalCoherence_avg{year}.h5'

    !generate_mask.py {str(image1)} -m 0 -o image1_mask.h5
    !generate_mask.py {str(image2)} -m 0 -o image2_mask.h5
    !add.py image1_mask.h5 image2_mask.h5 -o maskAdd.h5
    !generate_mask.py maskAdd.h5 -m 2 -o mask.h5
    
    !add.py {image1} {image2} -o tempCohAdd.h5
    !image_math.py tempCohAdd.h5 '/' 2 -o tempCohAvg.h5
    !mask.py tempCohAvg.h5 -m mask.h5 -o {out_name}

!rm tempCohAdd.h5 image1_mask.h5 image2_mask.h5 maskAdd.h5 tempCohAvg.h5 mask.h5

## Check Results

In [ ]:
dset = 'temporalCoherence_avg2019.h5'

In [ ]:
view.main(f'{dset} --noverbose'.split(' '))

In [ ]:
!info.py {dset}

## Export to Geotiffs

In [ ]:
!ls

In [ ]:
vels = list(Path().glob('velocity_*[ascdesc]20*.h5'))
vels_proj = list(Path().glob('velocity_*[ewup]20*.h5'))
cohs = list(Path().glob('temporalCoherence_*[ascdesc]20*.h5'))
cohs_proj = list(Path().glob('temporalCoherence_avg20*.h5'))
stds = list(Path().glob('std_*[ascdesc]20*.h5'))
stds_proj = list(Path().glob('std_*[ewup]20*.h5'))

# files = vels + cohs
files = vels + vels_proj + cohs + cohs_proj + stds + stds_proj

files = stds

In [ ]:
for f in files:
    outfile = str(Path('../results') / f.with_suffix('.tif'))
    !save_gdal.py {str(f)} -o {outfile}

In [ ]:
from_file = list(Path('..').glob('./*/coherenceSpatialAvg*txt'))
to_file = [Path('../results') / f'{x.parent.name}_{x.name}' for x in from_file]

for f,t in zip(from_file,to_file):
    !cp {str(f)} {str(t)}

## Done!

In [ ]:
!ls *mask*h5

In [ ]:
!rm *mask*h5